In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from collections import defaultdict
import matplotlib.pyplot as plt
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.word2vec import Word2Vec
%matplotlib inline

In [2]:
s_trans = pd.read_csv("sorted_transactions.csv")
customers_gender = pd.read_csv("customers_gender_train.csv")
mc_codes = pd.read_csv("tr_mcc_codes.csv", sep=";")

In [6]:
s_trans

,Unnamed: 0,index,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,hour
0,0,1453475,6815,10 10:52:00,4814,1030,-2245.92,NaN,250
1,1,1453476,6815,10 14:44:36,4814,1030,-2245.92,NaN,254
2,2,1453477,6815,11 10:23:14,6010,7031,2470507.35,NaN,274
3,3,1453478,6815,14 12:49:46,6011,2010,-11229.58,NaN,348
4,4,1453479,6815,17 12:33:31,4814,1030,-2245.92,NaN,420
5,5,1453480,6815,17 14:14:38,6011,2010,-44918.32,NaN,422
6,6,1453481,6815,18 14:35:17,6011,2010,-44918.32,NaN,446
7,7,1453482,6815,23 11:45:37,4814,1030,-2245.92,NaN,563
8,8,1453483,6815,32 11:12:59,6011,2010,-22459.16,NaN,779
9,9,1453484,6815,32 11:56:03,6011,2010,-22459.16,NaN,779


In [3]:
def trans_sort(transactions, hours = 12, out_style = "int_list"):
    from IPython.display import clear_output
    '''
    transaction - transaction.csv
    hours - По каким промежутка времени он будет делить mcc_codes
    out_style = "str_list" или "int_list"
    '''
    if out_style == "int_list":
        int_list = defaultdict(list)
        idd = 0
        int_list[0].append(transactions["mcc_code"][0])
        for i in range(1, len(transactions)):
            if transactions["customer_id"][i] != transactions["customer_id"][i - 1]:
                idd = idd + 1
            int_list[idd].append((transactions["mcc_code"][i]))
            if (i % 68490 == 0):
                clear_output()
                print(int(i / 68490), "%", sep='')
        return int_list
    if out_style == "str_list":
        str_list = []
        strr = []
        timm = []
        idd = 0
        j = 0
        for i in range(1, len(transactions)):
            if len(timm) > 0 and transactions["hour"][i] > timm[0] + hours:
                str_list.append(strr)
                j = 0
                strr = []
                timm = []
            if len(timm) > 0 and transactions["customer_id"][i] != transactions["customer_id"][i - 1]:
                str_list.append(strr)
                j = 0
                strr = []
                timm = []
            strr.append(str(transactions["mcc_code"][i]))
            timm.append(transactions["hour"][i])
            if (i % 68490 == 0):
                clear_output()
                print(int(i / 68490), "%", sep='')
        return str_list

In [4]:
trams = trans_sort(transactions=s_trans)

98%


KeyboardInterrupt: 

In [6]:
trams_1 = trans_sort(transactions=s_trans, out_style='str_list')

100%


In [18]:
trams_1

[['4814', '6010'],
 ['6011'],
 ['4814', '6011'],
 ['6011'],
 ['4814'],
 ['6011', '6011'],
 ['4814', '6011', '4814'],
 ['4814', '6011'],
 ['6011'],
 ['4814', '4814'],
 ['6011', '6011'],
 ['6011', '4814', '4814'],
 ['6011'],
 ['6011'],
 ['6011'],
 ['6011', '4814'],
 ['4814', '6011', '6011'],
 ['6011'],
 ['6011', '4814', '6011'],
 ['4814'],
 ['4814', '4814'],
 ['4814'],
 ['4814'],
 ['6011', '6011'],
 ['4814'],
 ['4814'],
 ['4814'],
 ['4814'],
 ['6011'],
 ['6011'],
 ['6011'],
 ['4814'],
 ['6011'],
 ['6011'],
 ['4814', '6011'],
 ['4814'],
 ['6011', '6011', '4814'],
 ['6011'],
 ['6011'],
 ['4814', '6011'],
 ['4814'],
 ['4814', '4814'],
 ['6011'],
 ['6011', '4814'],
 ['6011', '4814'],
 ['6011'],
 ['6011'],
 ['6011'],
 ['6011'],
 ['6011', '4814', '5411'],
 ['4814', '5311', '6011', '6011', '4814'],
 ['6011'],
 ['4814', '6011'],
 ['6011'],
 ['4814'],
 ['5311', '4814'],
 ['4814'],
 ['4814', '6011'],
 ['5411', '6011', '5311'],
 ['5411'],
 ['6011', '5411', '5411', '5311'],
 ['5311'],
 ['4814'],
 ['

In [9]:
model = Word2Vec(trams_1, size=100, window=5, min_count=5, workers=4)

In [11]:
model.save("mcc2vec")

In [13]:
print(model)

Word2Vec(vocab=184, size=100, alpha=0.025)
